In [30]:
import pandas as pd
import os
from urllib.parse import unquote
import datetime
import numpy as np
import ast


os.chdir("/home/gabri/Desktop/ADA/ADA_wikispeedia")
print(os.getcwd())

SAVE_FOLDER = "data_processed"

/home/gabri/Desktop/ADA/ADA_wikispeedia


# Article Name tsv

In [31]:
df_names_path = "data/wikispeedia_paths-and-graph/articles.tsv"
df_names = pd.read_csv(df_names_path, sep='\t', header=None, comment='#')

# name a colum
df_names = df_names.rename(columns={0:"article_name"})

# Decode the article name
df_names.article_name = df_names.article_name.apply(unquote)

# Use " " instead of "_"
df_names.article_name = df_names.article_name.str.replace('_', ' ', regex=False)

#  Reset row index
df_names.reset_index(drop=True, inplace=True)

# put ID column
df_names["article_id"] = df_names.index.astype(str)

# Save processed df
df_names.to_csv(os.path.join(SAVE_FOLDER, "articles_processed.csv"), index = False)

df_names

,article_name,article_id
0,Áedán mac Gabráin,0
1,Åland,1
2,Édouard Manet,2
3,Éire,3
4,Óengus I of the Picts,4
...,...,...
4599,Zionism,4599
4600,Zirconium,4600
4601,Zoroaster,4601
4602,Zuid-Gelders,4602


# Article Category tsv

In [32]:
df_categories_path = "data/wikispeedia_paths-and-graph/categories.tsv"
df_categories = pd.read_csv(df_categories_path, sep='\t', header=None, comment='#')

# name a colum
df_categories = df_categories.rename(columns={0:"article_name", 1:"article_category"})

# Decode the article name
df_categories.article_name = df_categories.article_name.apply(unquote)

# Use " " instead of "_"
df_categories.article_name = df_categories.article_name.str.replace('_', ' ', regex=False)

# Use " " instead of "_"
df_categories.article_category = df_categories.article_category.str.replace('_', ' ', regex=False)

# Split the article_category into a list of categories
df_categories['article_category_list'] = df_categories['article_category'].str.split('.')

# Remove 'subject' from each list
df_categories['article_category_list'] = df_categories['article_category_list'].apply(lambda x: [cat for cat in x if cat.strip() != "subject"])

# Create a col for each cat
category_lists = df_categories['article_category_list']
category_df = pd.DataFrame(category_lists.tolist())
category_df.columns = [f'article_category_{i+1}' for i in range(category_df.shape[1])]
df_categories = pd.concat([df_categories, category_df], axis=1)


# HOW TO DEAL WITH ARTICLE THAT HAVE MORE CATEGORIES???
#print("Article with same names:")
#print(df_categories[df_categories["article_name"].duplicated()])
# ??????????????????????''
# ??????????????????????

#  Reset row index
df_categories.reset_index(drop=True, inplace=True)

display(df_categories)

df_categories.to_csv(os.path.join(SAVE_FOLDER, "categories_processed.csv"), index = False)


,article_name,article_category,article_category_list,article_category_1,article_category_2,article_category_3
0,Áedán mac Gabráin,subject.History.British History.British Histor...,"[History, British History, British History 150...",History,British History,British History 1500 and before including Roma...
1,Áedán mac Gabráin,subject.People.Historical figures,"[People, Historical figures]",People,Historical figures,None
2,Åland,subject.Countries,[Countries],Countries,None,None
3,Åland,subject.Geography.European Geography.European ...,"[Geography, European Geography, European Count...",Geography,European Geography,European Countries
4,Édouard Manet,subject.People.Artists,"[People, Artists]",People,Artists,None
...,...,...,...,...,...,...
5199,Zirconium,subject.Science.Chemistry.Chemical elements,"[Science, Chemistry, Chemical elements]",Science,Chemistry,Chemical elements
5200,Zoroaster,subject.People.Religious figures and leaders,"[People, Religious figures and leaders]",People,Religious figures and leaders,None
5201,Zuid-Gelders,subject.Geography.European Geography,"[Geography, European Geography]",Geography,European Geography,None
5202,Zuid-Gelders,subject.Language and literature.Languages,"[Language and literature, Languages]",Language and literature,Languages,None


# Article links relationship

In [33]:
df_links_path = "data/wikispeedia_paths-and-graph/links.tsv"
df_links = pd.read_csv(df_links_path, sep='\t', header=None, comment='#')

# name a colum
df_links = df_links.rename(columns={0:"link_source", 1:"link_target"})

# Decode the article name and Use " " instead of "_"
df_links.link_source = df_links.link_source.apply(unquote)
df_links.link_source = df_links.link_source.str.replace('_', ' ', regex=False)

df_links.link_target = df_links.link_target.apply(unquote)
df_links.link_target = df_links.link_target.str.replace('_', ' ', regex=False)

# add coplumns with article Id
df_article = pd.read_csv(os.path.join(SAVE_FOLDER, "articles_processed.csv"))

    #soruce
df_links = pd.merge(left = df_links, 
                    right = df_article,
                    left_on="link_source",
                    right_on="article_name")
df_links = df_links.rename(columns={"article_id": "link_source_id"})
df_links.drop(["article_name"], axis=1, inplace=True)

    #target
df_links = pd.merge(left = df_links, 
                    right = df_article,
                    left_on="link_target",
                    right_on="article_name")

df_links = df_links.rename(columns={"article_id": "link_target_id"})
df_links.drop(["article_name"], axis=1, inplace=True)

df_links.to_csv(os.path.join(SAVE_FOLDER, "links_processed.csv"), index = False)

df_links


,link_source,link_target,link_source_id,link_target_id
0,Áedán mac Gabráin,Bede,0,530
1,Áedán mac Gabráin,Columba,0,974
2,Áedán mac Gabráin,Dál Riata,0,1115
3,Áedán mac Gabráin,Great Britain,0,1772
4,Áedán mac Gabráin,Ireland,0,2149
...,...,...,...,...
119877,Zulu,South Africa,4603,3803
119878,Zulu,Swaziland,4603,3967
119879,Zulu,United Kingdom,4603,4293
119880,Zulu,Zambia,4603,4587


# Path Finished


In [34]:
df_pf_path = "data/wikispeedia_paths-and-graph/paths_finished.tsv"
df_pf = pd.read_csv(df_pf_path, sep='\t', header=None, comment='#')

# name a colum
df_pf = df_pf.rename(columns={0:"hashed_ip", 1:"timestamp", 2:"duration", 3:"path", 4:"rating"})

# create a list of liks for rach path
df_pf['path_list'] = df_pf['path'].str.split(';')

# for each element of the path: 1) decode, 2) substitue "_" with " "
df_pf['path_list'] = df_pf['path_list'].apply(lambda links: [unquote(link.replace('_', ' ')) for link in links])

# source link and target link
df_pf['source_link'] = df_pf['path_list'].apply(lambda x: x[0] if len(x) > 0 else None)  # First element
df_pf['target_link'] = df_pf['path_list'].apply(lambda x: x[-1] if len(x) > 0 else None)  # Last element

# colum for finished or not
df_pf["finished"] = True

# column for cause fo unfinisehd in case (None for finished paths)
df_pf["type_unfinished"] = None

df_pf.head(2)

,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[14th century, 15th century, 16th century, Pac...",14th century,African slave trade,True,None
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[14th century, Europe, Africa, Atlantic slave ...",14th century,African slave trade,True,None


# Path Unfinished

In [35]:
df_uf_path = "data/wikispeedia_paths-and-graph/paths_unfinished.tsv"
df_uf = pd.read_csv(df_uf_path, sep='\t', header=None, comment='#')

# name a colum
df_uf = df_uf.rename(columns={0:"hashed_ip", 1:"timestamp", 2:"duration", 3:"path", 4:"target_link", 5:"type_unfinished"})

# create a list of liks for rach path
df_uf['path_list'] = df_uf['path'].str.split(';')

# for each element of the path: 1) decode, 2) substitue "_" with " "
df_uf['path_list'] = df_uf['path_list'].apply(lambda links: [unquote(link.replace('_', ' ')) for link in links])

# source link and target link
df_uf['source_link'] = df_uf['path_list'].apply(lambda x: x[0] if len(x) > 0 else None)  # First element

# colum for finished or not
df_uf["finished"] = False

# Rating columns (not present in unfinsef
df_uf["rating"] = None

# Remove usless cols

df_uf.head(5)

,hashed_ip,timestamp,duration,path,target_link,type_unfinished,path_list,source_link,finished,rating
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout,[Obi-Wan Kenobi],Obi-Wan Kenobi,False,None
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout,[Julius Caesar],Julius Caesar,False,None
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout,"[Malawi, Democracy, Alexander the Great]",Malawi,False,None
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart,[Paraguay],Paraguay,False,None
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout,"[Paraguay, Bolivia]",Paraguay,False,None


# Merge

In [36]:
# Put cols in the same order
cols_name = df_pf.columns.to_list()
df_up = df_uf[cols_name]

# Syack veticllay the 2 df
df_p = pd.concat([df_pf, df_up], axis=0, ignore_index=True)

df_p

/tmp/ipykernel_1343783/493869391.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_p = pd.concat([df_pf, df_up], axis=0, ignore_index=True)


,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[14th century, 15th century, 16th century, Pac...",14th century,African slave trade,True,None
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[14th century, Europe, Africa, Atlantic slave ...",14th century,African slave trade,True,None
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN,"[14th century, Niger, Nigeria, British Empire,...",14th century,African slave trade,True,None
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN,"[14th century, Renaissance, Ancient Greece, Gr...",14th century,Greece,True,None
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0,"[14th century, Italy, Roman Catholic Church, H...",14th century,John F. Kennedy,True,None
...,...,...,...,...,...,...,...,...,...,...
76188,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,NaN,"[Franz Kafka, Tuberculosis, World Health Organ...",Franz Kafka,Cholera,False,restart
76189,232f992e57d43e8d,1389787697,6,Modern_history,NaN,[Modern history],Modern history,Hollandic,False,restart
76190,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,NaN,"[Computer programming, Linguistics, Culture, P...",Computer programming,The_Beatles,False,timeout
76191,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,NaN,"[Jamaica, United Kingdom, World War II, Battle...",Jamaica,Alan_Turing,False,timeout


In [37]:
# Add other metrics

# number cliks
df_p['n_click'] = df_p['path_list'].apply(len)

# Count occurrences of "<" in each list of path_list
df_p['n_back'] = df_p['path_list'].apply(lambda x: len([el for el in x if el == "<"]))

#convert timestamp into date
df_p['date'] = df_p['timestamp'].apply(datetime.datetime.fromtimestamp)

# click rate
df_p['click_rate'] = df_p['n_click']/df_p['duration']
df_p['normalized_duration'] = df_p['duration'] / df_p['n_click']
# Replace infinite values with 0
df_p['click_rate'].replace([np.inf, -np.inf], 0, inplace=True)

# freq back click
df_p['freq_back'] = df_p['n_back']/df_p['n_click']

df_p


/tmp/ipykernel_1343783/503459126.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_p['click_rate'].replace([np.inf, -np.inf], 0, inplace=True)


,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished,n_click,n_back,date,click_rate,normalized_duration,freq_back
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[14th century, 15th century, 16th century, Pac...",14th century,African slave trade,True,None,9,0,2011-02-15 04:26:49,0.054217,18.444444,0.000
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[14th century, Europe, Africa, Atlantic slave ...",14th century,African slave trade,True,None,5,0,2012-08-12 08:36:52,0.056818,17.600000,0.000
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN,"[14th century, Niger, Nigeria, British Empire,...",14th century,African slave trade,True,None,8,0,2012-10-03 23:10:40,0.057971,17.250000,0.000
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN,"[14th century, Renaissance, Ancient Greece, Gr...",14th century,Greece,True,None,4,0,2010-02-08 08:25:25,0.108108,9.250000,0.000
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0,"[14th century, Italy, Roman Catholic Church, H...",14th century,John F. Kennedy,True,None,7,0,2013-04-23 17:27:08,0.040000,25.000000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76188,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,NaN,"[Franz Kafka, Tuberculosis, World Health Organ...",Franz Kafka,Cholera,False,restart,8,1,2014-01-15 13:06:45,0.044444,22.500000,0.125
76189,232f992e57d43e8d,1389787697,6,Modern_history,NaN,[Modern history],Modern history,Hollandic,False,restart,1,0,2014-01-15 13:08:17,0.166667,6.000000,0.000
76190,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,NaN,"[Computer programming, Linguistics, Culture, P...",Computer programming,The_Beatles,False,timeout,5,1,2014-01-15 16:06:40,0.002632,380.000000,0.200
76191,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,NaN,"[Jamaica, United Kingdom, World War II, Battle...",Jamaica,Alan_Turing,False,timeout,4,0,2014-01-15 16:24:41,0.002102,475.750000,0.000


# Encode paths with article_id

In [ ]:

# Convert string representation of lists to actual lists
#import ast
#df_p['path_list'] = df_p['path_list'].apply(ast.literal_eval)

#df_p.path_list.head(4)

In [39]:
# df for path name trnaspltion
df_names = pd.read_csv("data_processed/articles_processed.csv")

#Create a mapping dictionary from the DataFrame
mapping_dict = pd.Series(df_names.article_id.values, index=df_names.article_name).to_dict()
mapping_dict.update({"<":"<"})
#print(mapping_dict)


mapped_paths = []
for i, row in df_p.iterrows():
    path = row["path_list"]
    #print(path)
    mapped_path = [mapping_dict[item] for item in path]  
    #print(mapped_path)
    #print(i)
    mapped_paths.append(mapped_path)


df_p["path_list_id"] = mapped_paths

df_p

,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished,n_click,n_back,date,click_rate,normalized_duration,freq_back,path_list_id
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[14th century, 15th century, 16th century, Pac...",14th century,African slave trade,True,None,9,0,2011-02-15 04:26:49,0.054217,18.444444,0.000,"[10, 12, 15, 3134, 377, 105, 128, 379, 143]"
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[14th century, Europe, Africa, Atlantic slave ...",14th century,African slave trade,True,None,5,0,2012-08-12 08:36:52,0.056818,17.600000,0.000,"[10, 1433, 128, 379, 143]"
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN,"[14th century, Niger, Nigeria, British Empire,...",14th century,African slave trade,True,None,8,0,2012-10-03 23:10:40,0.057971,17.250000,0.000,"[10, 2982, 2984, 694, 3755, 128, 379, 143]"
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN,"[14th century, Renaissance, Ancient Greece, Gr...",14th century,Greece,True,None,4,0,2010-02-08 08:25:25,0.108108,9.250000,0.000,"[10, 3464, 241, 1793]"
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0,"[14th century, Italy, Roman Catholic Church, H...",14th century,John F. Kennedy,True,None,7,0,2013-04-23 17:27:08,0.040000,25.000000,0.000,"[10, 2183, 3529, 1836, 3542, 3342, 2266]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76188,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,NaN,"[Franz Kafka, Tuberculosis, World Health Organ...",Franz Kafka,Cholera,False,restart,8,1,2014-01-15 13:06:45,0.044444,22.500000,0.125,"[1585, 4247, 4538, 2614, 3223, <, 3917, 1696]"
76189,232f992e57d43e8d,1389787697,6,Modern_history,NaN,[Modern history],Modern history,Hollandic,False,restart,1,0,2014-01-15 13:08:17,0.166667,6.000000,0.000,[2802]
76190,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,NaN,"[Computer programming, Linguistics, Culture, P...",Computer programming,The_Beatles,False,timeout,5,1,2014-01-15 16:06:40,0.002632,380.000000,0.200,"[1008, 2478, 1098, 3315, <]"
76191,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,NaN,"[Jamaica, United Kingdom, World War II, Battle...",Jamaica,Alan_Turing,False,timeout,4,0,2014-01-15 16:24:41,0.002102,475.750000,0.000,"[2205, 4293, 4542, 506]"


In [40]:
def resolve_backsteps(path):
    resolved_path = []  # Final resolved path
    resolved_path_stack = []  # Stack to track the unique articles added

    for item in path:
        if item != "<": # normal behavior
            resolved_path.append(item)
            resolved_path_stack.append(item)
        if item == "<":
                resolved_path.append(resolved_path_stack[-2]) # not last (beomse is the same)
                resolved_path_stack = resolved_path_stack[:-2] #remove the last 2 elemts

    return resolved_path

def resolve_backsteps(path):
    resolved_path = []  # Final resolved path
    resolved_path_stack = []  # Stack to track the unique articles added
    is_first_pop = True  # To track the first backstep in a sequence of backsteps

    for item in path:
        if item != "<":  # Normal behavior, add article
            resolved_path.append(item)
            resolved_path_stack.append(item)
            is_first_pop = True  # Reset backtracking flag when we encounter a new article
        elif item == "<" and is_first_pop:  # Handle the first backtracking step
            if len(resolved_path_stack) > 1:
                resolved_path_stack.pop()  # Remove the most recent article
                resolved_path.append(resolved_path_stack.pop())  # Append the second last article
            is_first_pop = False  # Mark that the first backstep has been handled
        elif item == "<" and not is_first_pop:  # Handle consecutive backsteps
            if resolved_path_stack:
                resolved_path.append(resolved_path_stack.pop())  # Simply pop the last unique article

    return resolved_path


# Apply the function to create a new column 'resolved_path_list_id'
df_p['resolved_path_list_id'] = df_p['path_list_id'].apply(resolve_backsteps)
# Save as array iof int
#df_p['resolved_path_list_id'] = df_p['resolved_path_list_id'].apply(lambda x: np.array(x, dtype=int))



# Display the DataFrame
display(df_p[['path_list_id', 'resolved_path_list_id']].head(20))

# Save
df_p.to_csv(os.path.join(SAVE_FOLDER, "all_articles_processed.csv"), index = False)

,path_list_id,resolved_path_list_id
0,"[10, 12, 15, 3134, 377, 105, 128, 379, 143]","[10, 12, 15, 3134, 377, 105, 128, 379, 143]"
1,"[10, 1433, 128, 379, 143]","[10, 1433, 128, 379, 143]"
2,"[10, 2982, 2984, 694, 3755, 128, 379, 143]","[10, 2982, 2984, 694, 3755, 128, 379, 143]"
3,"[10, 3464, 241, 1793]","[10, 3464, 241, 1793]"
4,"[10, 2183, 3529, 1836, 3542, 3342, 2266]","[10, 2183, 3529, 1836, 3542, 3342, 2266]"
5,"[10, 1433, 3011, 4297, 3342, 2266]","[10, 1433, 3011, 4297, 3342, 2266]"
6,"[10, 894, 1820, 1511]","[10, 894, 1820, 1511]"
7,"[10, 4147, 2165, 2466, 969, 3425]","[10, 4147, 2165, 2466, 969, 3425]"
8,"[10, 4147, 2466, 3425]","[10, 4147, 2466, 3425]"
9,"[10, 12, 3267, 2933, 4285, 969, 3425]","[10, 12, 3267, 2933, 4285, 969, 3425]"


# Nan

In [41]:
# Cehck for nan values, after reomobn colsumn thata re not presdent in both df
missing_rows = df_p[df_p.drop(["rating", "type_unfinished"], axis=1).isnull().any(axis=1)]
missing_rows

,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished,n_click,n_back,date,click_rate,normalized_duration,freq_back,path_list_id,resolved_path_list_id
10377,NaN,1219094972,96,Heat;Latin;Europe;Italy,NaN,"[Heat, Latin, Europe, Italy]",Heat,Italy,True,None,4,0,2008-08-18 23:29:32,0.041667,24.000000,0.00,"[1879, 2417, 1433, 2183]","[1879, 2417, 1433, 2183]"
13719,NaN,1219094972,140,Tyrannosaurus;London;Temperate;Weather;Solar_S...,NaN,"[Tyrannosaurus, London, Temperate, Weather, So...",Tyrannosaurus,Sun,True,None,6,0,2008-08-18 23:29:32,0.042857,23.333333,0.00,"[4271, 2538, 4019, 4437, 3791, 3939]","[4271, 2538, 4019, 4437, 3791, 3939]"
15545,NaN,1219094972,106,Tropical_Storm_Bonnie_%282004%29;Gulf_of_Mexic...,NaN,"[Tropical Storm Bonnie (2004), Gulf of Mexico,...",Tropical Storm Bonnie (2004),Rhodium,True,None,12,3,2008-08-18 23:29:32,0.113208,8.833333,0.25,"[4225, 1818, <, 2931, 1888, 871, 381, 1345, <,...","[4225, 1818, 4225, 2931, 1888, 871, 381, 1345,..."


In [42]:
df_p[df_p["duration"] == 0]

,hashed_ip,timestamp,duration,path,rating,path_list,source_link,target_link,finished,type_unfinished,n_click,n_back,date,click_rate,normalized_duration,freq_back,path_list_id,resolved_path_list_id
5857,43f864c75b2571b5,1350587401,0,Lesotho,NaN,[Lesotho],Lesotho,Lesotho,True,None,1,0,2012-10-18 21:10:01,0.0,0.0,0.0,[2449],[2449]
9589,5590014e126275b4,1225696755,0,Moon,NaN,[Moon],Moon,Moon,True,None,1,0,2008-11-03 08:19:15,0.0,0.0,0.0,[2832],[2832]
9872,43f864c75b2571b5,1350587395,0,Coal,NaN,[Coal],Coal,Coal,True,None,1,0,2012-10-18 21:09:55,0.0,0.0,0.0,[946],[946]
20919,43f864c75b2571b5,1350587389,0,Pyramid,NaN,[Pyramid],Pyramid,Pyramid,True,None,1,0,2012-10-18 21:09:49,0.0,0.0,0.0,[3382],[3382]
23347,321e4b101c5b58ff,1225726660,0,Apple,NaN,[Apple],Apple,Apple,True,None,1,0,2008-11-03 16:37:40,0.0,0.0,0.0,[305],[305]
24326,43f864c75b2571b5,1350587377,0,Snow_Goose,NaN,[Snow Goose],Snow Goose,Snow Goose,True,None,1,0,2012-10-18 21:09:37,0.0,0.0,0.0,[3765],[3765]
30761,43f864c75b2571b5,1350587420,0,Royal_Navy,NaN,[Royal Navy],Royal Navy,Royal Navy,True,None,1,0,2012-10-18 21:10:20,0.0,0.0,0.0,[3557],[3557]
33832,43f864c75b2571b5,1350587367,0,Abel_Tasman,NaN,[Abel Tasman],Abel Tasman,Abel Tasman,True,None,1,0,2012-10-18 21:09:27,0.0,0.0,0.0,[88],[88]
34302,43f864c75b2571b5,1350587407,0,American_Samoa,NaN,[American Samoa],American Samoa,American Samoa,True,None,1,0,2012-10-18 21:10:07,0.0,0.0,0.0,[223],[223]
38927,43f864c75b2571b5,1350587383,0,Florence_Nightingale,NaN,[Florence Nightingale],Florence Nightingale,Florence Nightingale,True,None,1,0,2012-10-18 21:09:43,0.0,0.0,0.0,[1536],[1536]
